In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * (height//2) * (width//2), 128)
        self.fc2 = nn.Linear(128, 64)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * (height//2) * (width//2))
        x = torch.relu(self.fc1(x))
        return torch.relu(self.fc2(x))


In [ ]:
class LSTM(nn.Module):
    def __init__(self, cnn, lstm_hidden_size, num_layers, sequence_length):
        super(LSTM, self).__init__()
        self.cnn = cnn
        self.lstm = nn.LSTM(input_size=64, hidden_size=lstm_hidden_size, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(lstm_hidden_size, 1)  # Output layer for image prediction

    def forward(self, x):
        batch_size, seq_len, channels, height, width = x.shape
        cnn_out = []
        
        for t in range(seq_len):
            frame = x[:, t, :, :, :]  # Get each frame in the sequence
            cnn_out.append(self.cnn(frame))  # Extract spatial features using CNN
        
        cnn_out = torch.stack(cnn_out, dim=1)  # Stack CNN outputs for LSTM input
        lstm_out, _ = self.lstm(cnn_out)  # Pass through LSTM
        
        # Predict the next image (or frame)
        output = self.fc_out(lstm_out[:, -1, :])  # Use the last LSTM output for prediction
        return output
